In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.cluster import KMeans
from skleaern.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# make numbers easier to read on display
pd.options.display.float_format = '{:20.2f}'.format 

# all columns on output
pd.set_option('display.max_columns', 999)